In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
(x_train, y_train), (x_test,y_test) = datasets.cifar10.load_data()

In [ ]:
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

x_train = x_train / 255.0
x_test = x_test / 255.0



In [ ]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])


cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


cnn.fit(x_train, y_train, epochs=15)

Epoch 1/15
1563/1563 [==============================] - 25s 16ms/step - loss: 2.3028 - accuracy: 0.0994
Epoch 2/15
1563/1563 [==============================] - 24s 15ms/step - loss: 2.3028 - accuracy: 0.1000
Epoch 3/15
1563/1563 [==============================] - 23s 15ms/step - loss: 2.3028 - accuracy: 0.0963
Epoch 4/15
1563/1563 [==============================] - 23s 15ms/step - loss: 2.3028 - accuracy: 0.0983
Epoch 5/15
1563/1563 [==============================] - 23s 15ms/step - loss: 2.3028 - accuracy: 0.0994
Epoch 6/15
1563/1563 [==============================] - 23s 15ms/step - loss: 2.3027 - accuracy: 0.0991
Epoch 7/15
1563/1563 [==============================] - 23s 15ms/step - loss: 2.3028 - accuracy: 0.0966
Epoch 8/15
1563/1563 [==============================] - 27s 17ms/step - loss: 2.3028 - accuracy: 0.0985
Epoch 9/15
1563/1563 [==============================] - 24s 16ms/step - loss: 2.3028 - accuracy: 0.1008
Epoch 10/15
1563/1563 [==============================] - 23s 15m

In [ ]:
cnn.evaluate(x_test,y_test)

313/313 [==============================] - 2s 6ms/step - loss: 0.9222 - accuracy: 0.6909


[0.922234058380127, 0.6909000277519226]